# Objective

This run will continue from where run 7 left off. 

In [1]:
from game import Game
from ai import SplendorAI
from player import get_phase_parameters
from constants import *
from datetime import datetime
# from player import get_phase_parameters
import sys
from collections import defaultdict
from collections import Counter
from copy import deepcopy
from pympler import asizeof



/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# simply used to initialize players
base_game = Game(id=0, n_players=4)

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9858285570866306496
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9971620250
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14462960768645512634
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [4]:
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [5]:
# temperature will decrease over time
# bumping it up a tiny bit from run 7 pt 1
def calculate_temperature(round):
    if round > 5:
        return 2.8/(1 + round/2.5) /(1.5+round)
    else:
        return 1.8/(1 + round) /(1.5+round)

In [6]:
# constants to define; default
NETWORK_HYPERPARAMETERS = {
    # player input
    'player_funnel_layers': [15,12,10],
    'reserved_funnel_layers': [12,10,],
    'inject_reserved_at_player_funnel_index': 1, # 0 same as input, 1 = as first layer, etc.
    'card_funnel_layers': [12,12,8],
    # game input
    'game_funnel_layers': [15, 12, 10],
    'game_objective_funnel_layers': [10, 8],
    'game_card_funnel_layers': [15, 12, 10],
    # overall, slightly increased from default
    'main_dense_layers': [84, 36, 15], #this is when everything is combined

    # output layers
    # this does not include the win layer
    'output_layers': [
        {
            'name': 'Q1',
            'lag': 1,
            'score': 1,
            'discount': 0.1,
            'gems': 0.01,

        },
        {
            'name': 'Q3',
            'lag':  3,
            'score': 1,
            'discount': 0.1,
            'gems': 0,
        },
        {
            'name': 'Q5',
            'lag': 5,
            'score': 1,
            'discount': 0.05,
            'gems': 0,
        },
    ],
}

def get_phase_parameters(phase):
    """
    training will be divided into 5 phases

    """
    if phase==1:
        return {
            'Q1': 0.5,
            'Q3': 0.3,
            'Q5': 0.15,
            'win': 0.05,
        }
    elif phase==2:
        return {
            'Q1': 0.35,
            'Q3': 0.25,
            'Q5': 0.25,
            'win': 0.15,
        }
    elif phase==3:
        return {
            'Q1': 0.25,
            'Q3': 0.25,
            'Q5': 0.25,
            'win': 0.25,
        }
    elif phase==4:
        return {
            'Q1': 0.15,
            'Q3': 0.2,
            'Q5': 0.35,
            'win': 0.3,
        }
    elif phase==5:
        return {
            'Q1': 0.1,
            'Q3': 0.1,
            'Q5': 0.4,
            'win': 0.4,
        }
    elif phase==6:
        return {
            'Q1': 0.03,
            'Q3': 0.1,
            'Q5': 0.22,
            'win': 0.75
        }

In [7]:
players = base_game.players




In [8]:
# load previous models
#for i, player in enumerate(players):
#    player.ai.load_models('run7_player_%s_' % i, 5)

In [9]:
def summarize_counters(counter_list):
    master_counter = Counter()
    n = len(counter_list)
    for counter in counter_list:
        master_counter.update(counter)
    keys = list(master_counter.keys())
    for k in sorted(keys):
        master_counter[k] /= n
    return master_counter

In [10]:
set_durations = {}
game_data = defaultdict(list)
win_data = defaultdict(Counter)

In [11]:
# load AIs in advance

ai_objects = defaultdict(dict)
for round_id in [1,5,11]:
    print('loading round %d' % round_id)
    game = Game(id=-1, n_players=4)
    for i, player in enumerate(game.players):
        print('player %d' % i)
        player.ai.load_models('run7_player_%s_'  % i, round_id)
        ai_objects[round_id][i] = player.ai


# Test multiple runs of games

In [13]:
n_rounds = 6
round_start = 6
n_sets_per_round = 3
n_simulations_per_set = 320

start_time = datetime.now()
game_counter = 0



    
# compare rounds 1, 5, and 11
for pid in range(4):
    print('BASE PLAYER: %d' % pid)
    base_player = players[pid]
    other_players = [player for i, player in enumerate(players) if i != pid]
    for other_round in [1,5,11]:
        print('OTHER ROUND: %d' % other_round)
        
        other_phase = min(1, other_round // 2 + 1)
        other_temperature = calculate_temperature(other_round)
        for opid in range(4):
            if opid==pid:
                continue
            players[opid].ai = ai_objects[other_round][opid]
            players[opid].set_temperature(other_temperature)
            players[opid].decision_weighting = get_phase_parameters(other_phase)
       
      
        for base_round in [1,5,11]:
            print('BASE ROUND: %d' % base_round)
            round_tuple = (base_round, other_round, pid)
            base_phase = min(1, base_round // 2 + 1)
            base_temperature = calculate_temperature(base_round)
            base_player.ai = ai_objects[base_round][pid]
            base_player.set_temperature(base_temperature)
            base_player.decision_weighting = get_phase_parameters(base_phase)
            for gid in range(50):
                sys.stdout.write('.')
                sys.stdout.flush()
                game_counter+=1
                new_game = Game(id=game_counter, players=players)
                new_game.run()
                game_data[round_tuple].append(deepcopy(new_game.copy_plain_data_for_self_and_players()))
                win_data[round_tuple].update({i:1 for i in range(4) if players[i].win})
            print('/')
            # save historic data
            import pickle

            with open('run7_comparison_game_data.dat', 'wb') as f:
                pickle.dump(game_data, f)

            with open('run7_comparison_win_data.dat', 'wb') as f:
                pickle.dump(win_data, f)
                
            # for player in players:
            #    print(asizeof.asizeof(player.ai.win_model))

            
    
    
stop_time = datetime.now()
for time in [start_time, stop_time]:
    print(time.strftime('%x %X'))
    



BASE PLAYER: 0
OTHER ROUND: 1
BASE ROUND: 1
................................................../
BASE ROUND: 5
................................................../
BASE ROUND: 11
................................................../
OTHER ROUND: 5
BASE ROUND: 1
................................................../
BASE ROUND: 5
................................................../
BASE ROUND: 11
................................................../
OTHER ROUND: 11
BASE ROUND: 1
................................................../
BASE ROUND: 5
................................................../
BASE ROUND: 11
................................................../
BASE PLAYER: 1
OTHER ROUND: 1
BASE ROUND: 1
................................................../
BASE ROUND: 5
................................................../
BASE ROUND: 11
................................................../
OTHER ROUND: 5
BASE ROUND: 1
................................................../
BASE ROUND: 5
..................

In [ ]:
for player in players:
     print(asizeof.asizeof(player.ai.win_model))

In [ ]:
# players[0].ai.load_models('run6_player_0_round_', 1)

In [ ]:
from collections import Mapping, Container 
from sys import getsizeof

def deep_getsizeof(o, ids): 

    d = deep_getsizeof
    if id(o) in ids:
        return 0

    r = getsizeof(o)
    ids.add(id(o))

    if isinstance(o, str) or isinstance(0, bytes):
        return r
    
    if isinstance(o, np.ndarray):
        return r

    if isinstance(o, Mapping):
        return r + sum(d(k, ids) + d(v, ids) for k, v in o.items())

    if isinstance(o, Container):
        return r + sum(d(x, ids) for x in o)

    return r

deep_getsizeof(game_data, set())
# deep_getsizeof(players[0].extended_serialized_action_history[0], set())

# for k in list(locals().keys()):
#    v = locals()[k]
#    size = deep_getsizeof(v, set())
#    if size > 100000:
#        print(k, ':', size)

In [ ]:
# players[1].reset(reset_extended_history=True)
# for k in dir(players[1]):
#    v = getattr(players[1], k)
#    if isinstance(v, (list, dict)):
#        print(k, len(v))

In [ ]:
#new_game = Game(id=i*200 + j*100+1.1, players=players)

In [ ]:
#for k in dir(players[2]):
#    v = getattr(players[2], k)
#    if isinstance(v, (list, dict)):
#        print(k, len(v))

In [ ]:
#len(players[0].ai.extended_serialized_history)

# NOTES

Runs are complete. Data can be analyzed in a different environment. Tweaks may be made for future runs based on simulation results from loaded models.